In [1]:
from keras.applications.xception import Xception
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras import backend as K

/home/david/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [12]:
def get_model():
    # create the base pre-trained model
    base_model = Xception(weights='imagenet', include_top=False)
    # add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    # let's add a fully-connected layer
    x = Dense(1024, activation='relu')(x)
    # and a logistic layer with 185 classes
    predictions = Dense(185, activation='softmax')(x)
    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)

    # first: train only the top layers (which were randomly initialized)
    # i.e. freeze all convolutional Xception layers
    for layer in base_model.layers[:-1]:
        layer.trainable = False
    # compile the model (should be done *after* setting layers to non-trainable)
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    return model
    

In [13]:
model = get_model()

In [4]:
data_gen_args = dict(rotation_range=90.,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     zoom_range=0.2)
datagen = ImageDataGenerator(**data_gen_args)

image_generator = datagen.flow_from_directory(
    '../data/train/images/lab',
    class_mode='categorical',
    seed=1)


Found 13759 images belonging to 185 classes.


In [ ]:
model.load_weights('model-weights-52-dropout-25.h5')

In [ ]:
model.load_weights('model-weights.h5')

In [ ]:
model.fit_generator(
    image_generator,
    steps_per_epoch=500,
    epochs=10, verbose=1, workers=4, use_multiprocessing=True)

In [7]:
evaluate_generator = datagen.flow_from_directory(
        '../data/validate/images/lab',
        target_size=(299, 299),
        batch_size=16,
        class_mode='categorical')

# model.evaluate_generator(evaluate_generator, workers=4, use_multiprocessing=True)

Found 4679 images belonging to 185 classes.


In [5]:
def train(model, epochs=10):
    history = model.fit_generator(
        image_generator,
        steps_per_epoch=500,
        epochs=epochs,
        verbose=1, 
        workers=4,
        use_multiprocessing=True)
    return history
    
def evaluate(model):
    loss, accuracy = model.evaluate_generator(evaluate_generator, workers=4, use_multiprocessing=True)
    return accuracy

def train_and_evaluate(model, epochs=10):
    history = train(model, epochs)
    accuracy = evaluate(model)
    return history, accuracy
    

In [17]:
train_and_evaluate(model, epochs=3)

Epoch 1/3
500/500 [==============================] - 1223s 2s/step - loss: 1.1463 - acc: 0.6698
Epoch 2/3
500/500 [==============================] - 1389s 3s/step - loss: 1.1113 - acc: 0.6811
Epoch 3/3
500/500 [==============================] - 1357s 3s/step - loss: 1.1257 - acc: 0.6818


(<keras.callbacks.History at 0x7f1214ac2b38>, 0.5608035905171623)

In [ ]:
model.save_weights('model-weights-52-dropout-25.h5')

In [ ]:
model.save('model-dropout-25.h5')

In [ ]:
model.save_weights('model-weights.h5')

In [14]:
model.load_weights('model-weights.h5')

In [15]:
evaluate(model)

0.5509724300446279